In [1]:
%cd ../..

c:\Users\olarinoyem\Project\Modern-Time-Series-Forecasting-with-Python-2E


# Creating Baseline forecasts with NIXTLA

In [2]:
import numpy as np
import pandas as pd
import time
import os
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly_resampler
from itertools import cycle

pio.templates.default = "plotly_white"
import warnings
import humanize

from functools import partial
from statsforecast.core import StatsForecast
from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from statsforecast.models import (
    Naive,
    SeasonalNaive,
    HistoricAverage,
    WindowAverage,
    SeasonalWindowAverage,
    RandomWalkWithDrift,
    HoltWinters,
    ETS,
    AutoETS,
    AutoARIMA,
    ARIMA,
    AutoTheta,
    DynamicTheta,
    DynamicOptimizedTheta,
    Theta,
    OptimizedTheta,
    TBATS,
    AutoTBATS,
    MSTL

)
from datasetsforecast.losses import *
from src.utils.ts_utils import forecast_bias

import time
from src.utils import plotting_utils

from tqdm import tqdm 
np.random.seed(42)
tqdm.pandas()

c:\Users\olarinoyem\AppData\Local\miniconda3\envs\modern_ts_2E\Lib\site-packages\statsforecast\core.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
c:\Users\olarinoyem\AppData\Local\miniconda3\envs\modern_ts_2E\Lib\site-packages\statsforecast\utils.py:237: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  "ds": pd.date_range(start="1949-01-01", periods=len(AirPassengers), freq="M"),


In [3]:
# this makes it so that the outputs of the predict methods have the id as a column 
# instead of as the index
if 'NIXTLA_ID_AS_COL' in os.environ:
    del os.environ['NIXTLA_ID_AS_COL']
os.environ['NIXTLA_ID_AS_COL'] = '1'

In [4]:
os.makedirs("imgs/chapter_4", exist_ok=True)
preprocessed = Path("data/london_smart_meters/preprocessed")

In [5]:
def format_plot(fig, legends = None, xlabel="Time", ylabel="Value", title="", font_size=15):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t:  t.update(name = next(names)))
    fig.update_layout(
            autosize=False,
            width=900,
            height=500,
            title_text=title,
            title={
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            titlefont={
                "size": 20
            },
            legend_title = None,
            legend=dict(
                font=dict(size=font_size),
                orientation="h",
                yanchor="bottom",
                y=0.98,
                xanchor="right",
                x=1,
            ),
            yaxis=dict(
                title_text=ylabel,
                titlefont=dict(size=font_size),
                tickfont=dict(size=font_size),
            ),
            xaxis=dict(
                title_text=xlabel,
                titlefont=dict(size=font_size),
                tickfont=dict(size=font_size),
            )
        )
    return fig

In [6]:
import plotly.express as px

def plot_forecast(pred_df, forecast_columns, timestamp_col, forecast_display_names=None):
    if forecast_display_names is None:
        forecast_display_names = forecast_columns
    else:
        assert len(forecast_columns) == len(forecast_display_names)
    
    mask = ~pred_df[forecast_columns[0]].isnull()
    colors = [c.replace("rgb", "rgba").replace(")", ", <alpha>)") for c in px.colors.qualitative.Dark2]
    act_color = colors[0]
    colors = cycle(colors[1:])
    dash_types = cycle(["dash", "dot", "dashdot"])
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=pred_df[mask][timestamp_col], y=pred_df[mask]['energy_consumption'],
                             mode='lines', line=dict(color=act_color.replace("<alpha>", "0.3")),
                             name='Actual Consumption'))
    
    for col, display_col in zip(forecast_columns, forecast_display_names):
        fig.add_trace(go.Scatter(x=pred_df[mask][timestamp_col], y=pred_df.loc[mask, col],
                                 mode='lines', line=dict(dash=next(dash_types), color=next(colors).replace("<alpha>", "1")),
                                 name=display_col))
    return fig

In [7]:
#Readin the missing value imputed and train test split data
try:
    train_df = pd.read_parquet(preprocessed/"selected_blocks_train_missing_imputed.parquet")
    train_df = train_df[['LCLid',"timestamp","energy_consumption","frequency"]]
    val_df = pd.read_parquet(preprocessed/"selected_blocks_val_missing_imputed.parquet")
    val_df = val_df[['LCLid',"timestamp","energy_consumption","frequency"]]
    test_df = pd.read_parquet(preprocessed/"selected_blocks_test_missing_imputed.parquet")
    test_df = test_df[['LCLid',"timestamp","energy_consumption","frequency"]]
except FileNotFoundError:
    print(f"Warning: File not found in {preprocessed}. Ensure you've run '01-Setting up Experiment Harness.ipynb' in Chapter 04 and that the file path is correct.")

In [8]:
print("Min train_df Date: " , train_df.timestamp.min())
print("Max train_df Date: " , train_df.timestamp.max())
print("Min val_df Date: " , val_df.timestamp.min())
print("Max val_df Date: " , val_df.timestamp.max())
print("Min test_df Date: " , test_df.timestamp.min())
print("Max test_df Date: " , test_df.timestamp.max())

Min train_df Date:  2012-01-01 00:00:00
Max train_df Date:  2013-12-31 23:30:00
Min val_df Date:  2014-01-01 00:00:00
Max val_df Date:  2014-01-31 23:30:00
Min test_df Date:  2014-02-01 00:00:00
Max test_df Date:  2014-02-27 23:30:00


In [9]:
train_df = train_df[train_df.timestamp >'2012-01-01']
print("Min train_df Date: " , train_df.timestamp.min())

Min train_df Date:  2012-01-01 00:30:00


In [10]:
#picking a single time series from the dataset for illustration
freq = train_df.iloc[0]['frequency']
ts_train = train_df.loc[train_df.LCLid=="MAC000193", ['LCLid',"timestamp","energy_consumption"]]
ts_val = val_df.loc[val_df.LCLid=="MAC000193", ['LCLid',"timestamp","energy_consumption"]]
ts_test = test_df.loc[test_df.LCLid=="MAC000193", ['LCLid',"timestamp","energy_consumption"]]


# Baseline Forecasts

In [11]:
pred_df = pd.concat([ts_train, ts_val])
metric_df = pd.DataFrame([])


In [12]:
def evaluate_performace(
        ts_train, 
        ts_test, 
        models, 
        metrics,
        freq, 
        level,
        id_col,
        time_col,
        target_col,
        h):
    evaluation = {}
    results = ts_test.copy()

    # Timing dictionary to store train and predict durations
    timing = {}

    for model in models:
        model_name = model.__class__.__name__
        evaluation[model_name] = {}

        # Start the timer for fitting and prediction
        start_time = time.time()

        # Instantiate StatsForecast class
        sf = StatsForecast(
            models=[model],
            freq=freq,
            n_jobs=-1,
            fallback_model = Naive()
        )
        # Efficiently predict without storing memory
        y_pred = sf.forecast( 
                            h = h,
                            df=ts_train,        
                            id_col = id_col,
                            time_col = time_col,
                            target_col = target_col,
                            level = level,
                            )

        # Calculating the duration
        duration = time.time() - start_time
        timing[model_name] = duration

        # Merge multiple models to original dataframe
        results = results.merge(y_pred, how='left', on=[id_col, time_col])
        
        # # Calculate metrics
        for metric in metrics:
            metric_name = metric.__name__
            if metric_name == 'mase':
                evaluation[model_name][metric_name] = metric(results[target_col].values, 
                                                results[model_name].values, 
                                                ts_train[target_col].values, seasonality = 48)
            else:
                evaluation[model_name][metric_name] = metric(results[target_col].values, results[model_name].values)

        # # Add in total time for model fit
        evaluation[model_name]['Time Elapsed'] = timing[model_name]
        
        global metric_df
        temp_df = pd.DataFrame(evaluation).T.reset_index(drop = True)
        temp_df['Model'] = model_name
        temp_df = temp_df.set_index(['Model']).reset_index()
        metric_df = pd.concat([metric_df, temp_df]) 

    return results, metric_df

# NAIVE FORECAST

In [13]:
results, metrics = (
    evaluate_performace(
        ts_train, 
        ts_val, 
        models = [Naive()], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val)
        )
)


In [14]:
metrics

Model      mase       mae       mse      rmse      smape  forecast_bias  \
0  Naive  1.819549  0.305395  0.249304  0.499304  113.58813      74.340254   

   Time Elapsed  
0       0.05102

In [15]:
model_name = ['Naive']
model_display_name = ['Naive']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/naive.png")
fig.show()

# SEASONAL NAIVE FORECAST

In [16]:
results, metrics = (
    evaluate_performace(
        ts_train, 
        ts_val, 
        models = [SeasonalNaive(season_length=48*7)], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val)
        )
)


In [17]:
metrics

Model      mase       mae       mse      rmse       smape  \
0          Naive  1.819549  0.305395  0.249304  0.499304  113.588130   
0  SeasonalNaive  1.500943  0.251919  0.190827  0.436838   78.738832   

   forecast_bias  Time Elapsed  
0      74.340254      0.051020  
0      13.735403      0.043966

In [18]:
model_name = ['SeasonalNaive']
model_display_name = ['SeasonalNaive']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/seasonal_naive.png")
fig.show()

## Moving Average Forecast

In [19]:
results, metrics = (
    evaluate_performace(
        ts_train, 
        ts_val, 
        models = [WindowAverage(window_size = 48)], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val)
        )
)


In [20]:
metrics

Model      mase       mae       mse      rmse       smape  \
0          Naive  1.819549  0.305395  0.249304  0.499304  113.588130   
0  SeasonalNaive  1.500943  0.251919  0.190827  0.436838   78.738832   
0  WindowAverage  1.857255  0.311723  0.182946  0.427722  100.698209   

   forecast_bias  Time Elapsed  
0      74.340254      0.051020  
0      13.735403      0.043966  
0      11.509527      0.043003

In [21]:
model_name = ['WindowAverage']
model_display_name = ['WindowAverage']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/window_average.png")
fig.show()

# Exponential Smoothing Forecast

There are several forecast of exponential smoothing as discussed in the chapter.   NIXTLA allows you do specify the type of exponential smoothing you would like to do.  Or, you can use the ETS function which will test which version of exponential smoothing best fits your data and automatically finds the best parameters.

In [22]:
results, metrics = (
    evaluate_performace(
        ts_train, 
        ts_val, 
        models = [ HoltWinters(season_length = 48)], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val)
        )
)


In [23]:
metrics

Model      mase       mae       mse      rmse       smape  \
0          Naive  1.819549  0.305395  0.249304  0.499304  113.588130   
0  SeasonalNaive  1.500943  0.251919  0.190827  0.436838   78.738832   
0  WindowAverage  1.857255  0.311723  0.182946  0.427722  100.698209   
0    HoltWinters  1.138965  0.191165  0.100847  0.317565   88.889992   

   forecast_bias  Time Elapsed  
0      74.340254      0.051020  
0      13.735403      0.043966  
0      11.509527      0.043003  
0      10.440250     32.329801

In [24]:
model_name = ['HoltWinters']
model_display_name = ['HoltWinters']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/ets.png")
fig.show()

In [25]:
results, metrics = (
    evaluate_performace(
        ts_train, 
        ts_val, 
        models = [ AutoETS(model = 'AAA',season_length = 48)], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val)
        )
)


In [26]:
results.head()

LCLid           timestamp  energy_consumption   AutoETS
0  MAC000193 2014-01-01 00:00:00               0.223  0.207815
1  MAC000193 2014-01-01 00:30:00               0.274  0.033957
2  MAC000193 2014-01-01 01:00:00               0.308  0.014500
3  MAC000193 2014-01-01 01:30:00               0.279 -0.019726
4  MAC000193 2014-01-01 02:00:00               0.000 -0.016634

In [27]:
metrics

Model      mase       mae       mse      rmse       smape  \
0          Naive  1.819549  0.305395  0.249304  0.499304  113.588130   
0  SeasonalNaive  1.500943  0.251919  0.190827  0.436838   78.738832   
0  WindowAverage  1.857255  0.311723  0.182946  0.427722  100.698209   
0    HoltWinters  1.138965  0.191165  0.100847  0.317565   88.889992   
0        AutoETS  1.138965  0.191165  0.100847  0.317565   88.889992   

   forecast_bias  Time Elapsed  
0      74.340254      0.051020  
0      13.735403      0.043966  
0      11.509527      0.043003  
0      10.440250     32.329801  
0      10.440250     20.879981

In [28]:
model_name = ['AutoETS']
model_display_name = ['AutoETS']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/ets.png")
fig.show()

## ARIMA

Not using AutoARIMA because it just takes too much time for long time series
Sample AutoARIMA is given in this notebook.

In [29]:



results, metrics = (
    evaluate_performace(
        ts_train, 
        ts_val, 
        models = [ ARIMA(order = (2,1,1), seasonal_order = (1,1,1), season_length = 48)],
        #models = [ ARIMA(order = (0,1,2), seasonal_order = (0,0,2), season_length = 48)], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val)
        )
)

In [30]:
metrics

Model      mase       mae       mse      rmse       smape  \
0          Naive  1.819549  0.305395  0.249304  0.499304  113.588130   
0  SeasonalNaive  1.500943  0.251919  0.190827  0.436838   78.738832   
0  WindowAverage  1.857255  0.311723  0.182946  0.427722  100.698209   
0    HoltWinters  1.138965  0.191165  0.100847  0.317565   88.889992   
0        AutoETS  1.138965  0.191165  0.100847  0.317565   88.889992   
0          ARIMA  1.212629  0.203529  0.106482  0.326316   98.716784   

   forecast_bias  Time Elapsed  
0      74.340254      0.051020  
0      13.735403      0.043966  
0      11.509527      0.043003  
0      10.440250     32.329801  
0      10.440250     20.879981  
0      25.337937     17.785805

In [31]:
model_name = ['ARIMA']
model_display_name = ['ARIMA']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/ARIMA.png")
fig.show()

## Sample Auto ARIMA
Note:  This may take a long time to run.

To get a better understanding of what parameters to use, you can run AutoARIMA and output the fitted parameters.  AutoARIMA however is very slow, so I have includedd a sample output from AutoARIMA.

In [32]:
# sf = StatsForecast(
#     models=[AutoARIMA( max_p = 2, max_d=1, max_q = 2, max_P=2, max_D = 1, max_Q = 2, 
#                       start_p = 1, start_q = 1, start_P = 1, start_Q = 1, stepwise = True, season_length=48)],
#     freq=freq,
#     n_jobs=-1,
#     fallback_model = Naive()
# )

# y_pred = sf.fit( 
                  
#                     df=ts_train,        
#                     id_col = 'LCLid',
#                     time_col = 'timestamp',
#                     target_col = 'energy_consumption',

#                     )
# sf.fitted_[0,0].model_

#### Sample AutoARIMA Output

note the ARMA section
arma': (0, 2, 0, 2, 48, 1, 0)

This correspondes to:  p, q, P, Q, s, d, D 

p = 0, d = 1, q = 2

P = 0, D = 0, Q = 2

These parameters however could still be further tuned but due to the timing, will leave that as an exercise.  Better parameters include above in our ARIMA model.

In [33]:
# {'coef': {'ma1': -0.3800676288610244,
#   'ma2': -0.25121322485842584,
#   'sma1': 0.1300139905884248,
#   'sma2': 0.1045851891918472},
#  'sigma2': 0.05918752833035802,
#  'var_coef': array([[ 7.04752468e-07,  9.11329831e-07,  2.55693473e-07,
#          -1.87114423e-06],
#         [ 9.11329831e-07,  7.41876851e-06, -3.18464778e-07,
#          -8.00782484e-06],
#         [ 2.55693473e-07, -3.18464778e-07,  5.50841803e-07,
#          -4.88097177e-07],
#         [-1.87114423e-06, -8.00782484e-06, -4.88097177e-07,
#           1.03626760e-05]]),
#  'mask': array([ True,  True,  True,  True]),
#  'loglik': -188.2911979664168,
#  'aic': 386.5823959328336,
#  'arma': (0, 2, 0, 2, 48, 1, 0),
#  'residuals': array([ 0.000368  ,  0.01585512, -0.19137661, ..., -0.3180852 ,
#         -0.35435519, -0.1594367 ]),
#  'code': 2,
#  'n_cond': 0,
#  'nobs': 35087,
# ...
#  'bic': 428.9103257852313,
#  'aicc': 386.58410626036135,
#  'ic': None,
#  'xreg': None,
#  'x': array([0.368, 0.386, 0.17 , ..., 0.147, 0.111, 0.09 ], dtype=float32),
#  'lambda': None}

## Theta
THETA method in NIXTLA can be a little slow due to it running tests on which is the optimal theta parameter. 

In [34]:
results, metrics = (
    evaluate_performace(
        ts_train, 
        ts_val, 
        models = [ Theta(season_length =48, decomposition_type = 'additive' )], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val)
        )
)

In [35]:
model_name = ['Theta']
model_display_name = ['Theta']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/auto_theta.png")
fig.show()

In [36]:
metrics

Model      mase       mae       mse      rmse       smape  \
0          Naive  1.819549  0.305395  0.249304  0.499304  113.588130   
0  SeasonalNaive  1.500943  0.251919  0.190827  0.436838   78.738832   
0  WindowAverage  1.857255  0.311723  0.182946  0.427722  100.698209   
0    HoltWinters  1.138965  0.191165  0.100847  0.317565   88.889992   
0        AutoETS  1.138965  0.191165  0.100847  0.317565   88.889992   
0          ARIMA  1.212629  0.203529  0.106482  0.326316   98.716784   
0          Theta  1.428325  0.239731  0.167725  0.409542  104.389441   

   forecast_bias  Time Elapsed  
0      74.340254      0.051020  
0      13.735403      0.043966  
0      11.509527      0.043003  
0      10.440250     32.329801  
0      10.440250     20.879981  
0      25.337937     17.785805  
0      56.917018     71.949634

## TBATS
NIXTLA again offers TBATS and AutoTBATS.  AutoTBATS can be slow, so here we will just do TBATS. Similar to AutoARIMA, you can run a model using AutoTBATS to get a recommendation of good parameters to use.


In [37]:
# sf = StatsForecast(
#     models=[TBATS(seasonal_periods  = 48, use_trend=True, use_damped_trend=True)],
#     freq=freq,
#     n_jobs=2
# )

# y_pred = sf.fit( 
                  
#                     df=ts_train,        
#                     id_col = 'LCLid',
#                     time_col = 'timestamp',
#                     target_col = 'energy_consumption',

#                     )
# sf.fitted_[0,0].model_

In [38]:
# sf = StatsForecast(
#     models=[AutoTBATS(seasonal_periods  = 48, use_trend=True, use_damped_trend=True)],
#     freq=freq,
#     n_jobs=2
# )

# y_pred = sf.fit( 
                  
#                     df=ts_train,        
#                     id_col = 'LCLid',
#                     time_col = 'timestamp',
#                     target_col = 'energy_consumption',

#                     )
# sf.fitted_[0,0].model_

In [39]:
results, metrics = (
    evaluate_performace(
        ts_train, 
        ts_val, 
        models = [ TBATS(seasonal_periods  = 48)], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val)
        )
)

c:\Users\olarinoyem\AppData\Local\miniconda3\envs\modern_ts_2E\Lib\site-packages\statsforecast\tbats.py:944: UserWarning:

The time series is trend-stationary, disabling trend components.



In [40]:
metrics

Model      mase       mae       mse      rmse       smape  \
0          Naive  1.819549  0.305395  0.249304  0.499304  113.588130   
0  SeasonalNaive  1.500943  0.251919  0.190827  0.436838   78.738832   
0  WindowAverage  1.857255  0.311723  0.182946  0.427722  100.698209   
0    HoltWinters  1.138965  0.191165  0.100847  0.317565   88.889992   
0        AutoETS  1.138965  0.191165  0.100847  0.317565   88.889992   
0          ARIMA  1.212629  0.203529  0.106482  0.326316   98.716784   
0          Theta  1.428325  0.239731  0.167725  0.409542  104.389441   
0          TBATS  1.290473  0.216594  0.111659  0.334154   94.634706   

   forecast_bias  Time Elapsed  
0      74.340254      0.051020  
0      13.735403      0.043966  
0      11.509527      0.043003  
0      10.440250     32.329801  
0      10.440250     20.879981  
0      25.337937     17.785805  
0      56.917018     71.949634  
0      18.834032      7.200700

In [41]:
model_name = ['TBATS']
model_display_name = ['TBATS']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/auto_tbats.png")
fig.show()

In [42]:
metric_styled = metrics.reset_index(drop=True).style.format({
            "mae": "{:.3f}", 
            "mse": "{:.3f}", 
            "mase": "{:.3f}", 
            "rmse": "{:.3f}", 
            "smape": "{:.3f}",            
            "forecast_bias": "{:.2f}%"}).highlight_min(color='lightgreen', subset=["mae","mse","mase","rmse","smape","Time Elapsed"])
display(metric_styled)

## MSTL

In [43]:
results, metrics = (
    evaluate_performace(
        ts_train, 
        ts_val, 
        models = [ MSTL(season_length = 48)], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val)
        )
)

In [44]:
model_name = ['MSTL']
model_display_name = ['MSTL']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/auto_tbats.png")
fig.show()

Based on this sample test, the best performing models are TBATS, HoltWinters, and ETS.  Lets build that for all models. Since AutoETS is faster than HoltWinters, we will focus on that.

# VALIDATION SET
## Running Baseline Forecast for all consumers

In [45]:
ids = list(train_df.LCLid.unique()[:500]) # slicing dataframe for the sake of time.  May want to consider setting this low if working on a slower machine

train_df = train_df[train_df.LCLid.isin(ids)]
val_df = val_df[val_df.LCLid.isin(ids)]
test_df = test_df[test_df.LCLid.isin(ids)]

print("Length of validation Data: ", len(val_df[val_df.LCLid =='MAC000948'].LCLid))

Length of validation Data:  1488


### Forecasting for Validation Period for TBATS and ARIMA

In [46]:
validation_models =  [ AutoETS(model = 'AAA',season_length = 48), TBATS(seasonal_periods  = 48)]
#validation_models = [Naive()]
validation_models_names = [model.__class__.__name__ for model in validation_models]
metric_df = pd.DataFrame([])
h_val = 1488

In [47]:
baseline_val_pred_df, aggval_metrics = (
    evaluate_performace(
        train_df[["LCLid","timestamp","energy_consumption"]], 
        val_df[["LCLid","timestamp","energy_consumption"]], 
        models =validation_models, 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = h_val
        )
)

#### Evaluation of Baseline Forecast

In [ ]:
metric_styled = aggval_metrics.reset_index(drop=True).style.format({
            "mae": "{:.3f}", 
            "mse": "{:.3f}", 
            "mase": "{:.3f}", 
            "rmse": "{:.3f}", 
            "smape": "{:.3f}",            
            "forecast_bias": "{:.2f}%"}).highlight_min(color='lightgreen', subset=["mae","mse","mase","rmse","smape"])
display(metric_styled)

Here we are importing metrics meant to be able to efficiently cycle through each local forecast. We will rename them to not have any conflicts with our other error metrics.

In both the below validation and testing datasets, we will calculate both the local and global accuracy for each model type.

In [ ]:
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import rmse as rmse_local
from utilsforecast.losses import mae as mae_local
from utilsforecast.losses import mse as mse_local
from utilsforecast.losses import mase as mase_local

from functools import partial

In [ ]:
#train_df.loc[:.train_df.columns != 'energy_consumption')]
#cv_df.loc[:, cv_df.columns != 'cutoff']
#results.rename(columns = {'unique_id':'LCLid','ds':'timestamp','y':'energy_consumption'}, inplace = True)
fcst_mase = partial(mase_local, seasonality=48)

baseline_val_metrics_df = evaluate(baseline_val_pred_df, 
        metrics=[rmse_local, mae_local, mse_local,fcst_mase],  
        train_df = train_df,      
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption', 
        #models = ['ARIMA','AutoTBATS']
        )

In [ ]:
cols_to_exclude  = ['LCLid', 'metric']
val_columns = [col for col in baseline_val_metrics_df.columns if col not in cols_to_exclude]
val_columns


['AutoETS', 'TBATS']

In [ ]:
baseline_val_metrics_df = pd.melt(baseline_val_metrics_df, id_vars = ['LCLid','metric'],value_vars=val_columns,var_name='Algorithm' , value_name='value')
baseline_val_metrics_df = baseline_val_metrics_df.pivot_table(index = ['LCLid','Algorithm'], columns = 'metric', values = 'value').reset_index()
baseline_val_metrics_df.head()

C:\Users\tacke\AppData\Local\Temp\ipykernel_22412\3784551575.py:2: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior



metric      LCLid Algorithm       mae      mase       mse      rmse
0       MAC000061   AutoETS  0.050167  0.858688  0.005333  0.073030
1       MAC000061     TBATS  0.090263  1.544993  0.011232  0.105980
2       MAC000062   AutoETS  0.073287  0.822545  0.030241  0.173899
3       MAC000062     TBATS  0.071106  0.798071  0.030697  0.175205
4       MAC000066   AutoETS  0.067893  1.301094  0.016907  0.130025

In [ ]:
fig = px.histogram(baseline_val_metrics_df, 
                   x="mase", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MASE", ylabel="Probability Density", title="Distribution of MASE in the dataset")
fig.update_layout(xaxis_range=[0,1])
# fig.write_image("imgs/chapter_4/mase_dist.png")
fig.show()

In [ ]:
fig = px.histogram(baseline_val_metrics_df, 
                   x="mae", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=100, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MAE", ylabel="Probability Density", title="Distribution of MAE in the dataset")
fig.write_image("imgs/chapter_4/mae_dist.png")
fig.update_layout(xaxis_range=[0,1.1])
fig.show()

In [ ]:
fig = px.histogram(baseline_val_metrics_df, 
                   x="mse", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MSE", ylabel="Probability Density", title="Distribution of MSE in the dataset")
fig.update_layout(xaxis_range=[0,1])
fig.write_image("imgs/chapter_4/mse_dist.png")
fig.show()

#### Saving the Baseline Forecasts and Metrics

In [ ]:
os.makedirs("data/london_smart_meters/output", exist_ok=True)
output = Path("data/london_smart_meters/output")

In [ ]:
baseline_val_pred_df.to_pickle(output/"baseline_val_prediction_df.pkl")
baseline_val_metrics_df.to_pickle(output/"baseline_val_metrics_df.pkl")
aggval_metrics.to_pickle(output/"baseline_val_aggregate_metrics.pkl")

# TEST SET
### Forecasting for Test Period

In [ ]:
_train_df = pd.concat([train_df, val_df])
print("Length of training Data: ", len(test_df[test_df.LCLid =='MAC000948'].LCLid))
test_df.tail()

Length of training Data:  1296


LCLid           timestamp  energy_consumption frequency
33259  MAC004796 2014-02-27 21:30:00               0.244     30min
33260  MAC004796 2014-02-27 22:00:00               0.216     30min
33261  MAC004796 2014-02-27 22:30:00               0.211     30min
33262  MAC004796 2014-02-27 23:00:00               0.116     30min
33263  MAC004796 2014-02-27 23:30:00               0.441     30min

In [ ]:
test_models =  [ AutoETS(model = 'AAA',season_length = 48), TBATS(seasonal_periods  = 48)]
#test_models = [Naive()]
h_test = 1296

In [ ]:
sf = StatsForecast(
    models=test_models,
    freq=freq,
    n_jobs=-1,
    fallback_model= SeasonalNaive(season_length=48)
)

# sf.fit( df = _train_df[['timestamp', 'LCLid', 'energy_consumption']] ,    
#         id_col = 'LCLid',
#         time_col = 'timestamp',
#         target_col = 'energy_consumption',
#         )

# baseline_test_pred_df = sf.predict( h =1296 )

# Memory efficient predictions
baseline_test_pred_df = sf.forecast(df=_train_df, 
                                        h=h_test, 
                                        level=[],   
                                        id_col = 'LCLid',
                                        time_col = 'timestamp',
                                        target_col = 'energy_consumption',
        )
baseline_test_pred_df = pd.merge(baseline_test_pred_df, test_df[['timestamp', 'LCLid', 'energy_consumption']], on = ['LCLid','timestamp'], how = 'left')

In [ ]:
baseline_test_pred_df.head()

LCLid           timestamp   AutoETS  TBATS  energy_consumption
0  MAC000061 2014-02-01 00:00:00  0.056101  0.015               0.066
1  MAC000061 2014-02-01 00:30:00  0.039241  0.064               0.063
2  MAC000061 2014-02-01 01:00:00  0.024739  0.060               0.040
3  MAC000061 2014-02-01 01:30:00  0.022753  0.059               0.020
4  MAC000061 2014-02-01 02:00:00  0.023229  0.042               0.018

In [ ]:
baseline_test_metrics_df = evaluate(baseline_test_pred_df, 
        metrics=[rmse_local, mae_local, mse_local,fcst_mase],  
        train_df = _train_df[['timestamp', 'LCLid', 'energy_consumption']],      
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption'
        )



In [ ]:
baseline_test_metrics_df.head()

LCLid metric   AutoETS     TBATS
0  MAC000061   rmse  0.078926  0.103424
1  MAC000062   rmse  0.169500  0.166167
2  MAC000066   rmse  0.068317  0.062972
3  MAC000086   rmse  0.168439  0.186434
4  MAC000126   rmse  0.104435  0.105968

In [ ]:
test_columns = [col for col in baseline_test_metrics_df.columns if col not in cols_to_exclude]
test_columns

['AutoETS', 'TBATS']

In [ ]:
baseline_test_metrics_df = pd.melt(baseline_test_metrics_df, id_vars = ['LCLid','metric'],value_vars= test_columns, var_name='Algorithm' , value_name='value')
baseline_test_metrics_df = baseline_test_metrics_df.pivot_table(index = ['LCLid','Algorithm'], columns = 'metric', values = 'value').reset_index()
baseline_test_metrics_df.head()

C:\Users\tacke\AppData\Local\Temp\ipykernel_22412\1357591231.py:2: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior



metric      LCLid Algorithm       mae      mase       mse      rmse
0       MAC000061   AutoETS  0.058033  0.994184  0.006229  0.078926
1       MAC000061     TBATS  0.075398  1.291667  0.010697  0.103424
2       MAC000062   AutoETS  0.091553  1.020420  0.028730  0.169500
3       MAC000062     TBATS  0.075708  0.843818  0.027612  0.166167
4       MAC000066   AutoETS  0.044511  0.844105  0.004667  0.068317

In [ ]:
validation_models_names

['AutoETS', 'TBATS']

In [ ]:
from datasetsforecast.losses import *

agg_test_metrics= {} 

for model in validation_models_names:

    actual_series = baseline_test_pred_df['energy_consumption'].values
    pred_series = baseline_test_pred_df[model].values

    agg_test_metrics = {
        "Algorithm": model,
        "MAE": mae(actual_series, pred_series),
        "MSE": mse(actual_series, pred_series),
        "meanMASE": baseline_test_metrics_df.mase.mean(),
        "Forecast Bias": forecast_bias(baseline_test_pred_df['energy_consumption'].values,  baseline_test_pred_df[model].values),
    }
    agg_test_metrics.update(agg_test_metrics)

agg_test_metrics_df = pd.DataFrame([agg_test_metrics])
agg_test_metrics_df

Algorithm       MAE       MSE  meanMASE  Forecast Bias
0     TBATS  0.143499  0.095902  1.086202     -11.635356

### Evaluation of Baseline Forecast

In [ ]:
display(agg_test_metrics_df.style.format({"MAE": "{:.3f}", 
                          "MSE": "{:.3f}", 
                          "meanMASE": "{:.3f}", 
                          "Forecast Bias": "{:.2f}%"}).highlight_min(color='lightgreen', subset=["MAE","MSE","meanMASE"]))

In [ ]:
fig = px.histogram(baseline_test_metrics_df, 
                   x="mase", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MASE", ylabel="Probability Density", title="Distribution of MASE in the dataset")
fig.update_layout(xaxis_range=[0,10])
# fig.write_image("imgs/chapter_4/mase_dist.png")
fig.show()

In [ ]:
fig = px.histogram(baseline_test_metrics_df, 
                   x="mae", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=100, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MAE", ylabel="Probability Density", title="Distribution of MAE in the dataset")
# fig.write_image("imgs/chapter_4/mae_dist.png")
fig.update_layout(xaxis_range=[0,1.1])
fig.show()

In [ ]:
fig = px.histogram(baseline_test_metrics_df, 
                   x="mse", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MSE", ylabel="Probability Density", title="Distribution of MSE in the dataset")
fig.update_layout(xaxis_range=[0,1])
# fig.write_image("imgs/chapter_4/mse_dist.png")
fig.show()

### Saving the Baseline Forecasts and Metrics

In [ ]:
os.makedirs("data/london_smart_meters/output", exist_ok=True)
output = Path("data/london_smart_meters/output")

In [ ]:
baseline_test_pred_df.to_pickle(output/"baseline_test_prediction_df.pkl")
baseline_test_metrics_df.to_pickle(output/"baseline_test_metrics_df.pkl")
agg_test_metrics_df.to_pickle(output/"baseline_test_aggregate_metrics.pkl")